# Custom dataset
- 미니 배치
- 데이터 로드

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.__version__

'1.9.1+cu111'

In [2]:
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더
from torch.utils.data import Dataset

In [3]:
# 랜덤 시드(random seed)를 줍니다.
torch.manual_seed(1)

In [4]:
# 데이터
x_train  =  [[73,  80,  75], 
            [93,  88,  93], 
            [89,  91,  90], 
            [96,  98,  100],   
            [73,  66,  70]]
y_train  =  [[152],  [185],  [180],  [196],  [142]]
# x_train  =  torch.FloatTensor([[73,  80,  75], 
#                                [93,  88,  93], 
#                                [89,  91,  90], 
#                                [96,  98,  100],   
#                                [73,  66,  70]])  
# y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

In [5]:
# Dataset 상속
class CustomDataset(Dataset): 
  def __init__(self, x, y):
    self.x_data = x
    self.y_data = y

  # 총 데이터의 개수를 리턴
  def __len__(self): 
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx): 
    x = torch.FloatTensor(self.x_data[idx])
    y = torch.FloatTensor(self.y_data[idx])
    return x, y

### custom dataset -> dataloader
- 데이터셋을 만들었다면 데이터로더를 사용 가능합니다. 

- 데이터로더는 기본적으로 2개의 인자를 입력받는다. 
    - 하나는 데이터셋 
    - 미니 배치의 크기입니다. 
    
- 미니 배치의 크기는 통상적으로 2의 배수를 사용합니다.

In [6]:
dataset = CustomDataset(x_train, y_train)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [7]:
dataset

In [8]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1) # 다중 선형 회귀이므로 input_dim=3, output_dim=1.

    def forward(self, x):
        return self.linear(x)

In [10]:
model = MultivariateLinearRegressionModel()
print(list(model.parameters()))  # Initial random parameters

[Parameter containing:
tensor([[-0.5435,  0.3462, -0.1188]], requires_grad=True), Parameter containing:
tensor([0.2937], requires_grad=True)]


In [11]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 

In [12]:
# Learning using mini-batch from dataloader
nb_epochs = 20
for epoch in range(nb_epochs + 1):
  for batch_idx, samples in enumerate(dataloader):
    print(batch_idx)
    print(samples)
    x_train, y_train = samples
    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))
    

0
[tensor([[ 73.,  80.,  75.],
        [ 96.,  98., 100.]]), tensor([[152.],
        [196.]])]
Epoch    0/20 Batch 1/3 Cost: 40394.078125
1
[tensor([[89., 91., 90.],
        [73., 66., 70.]]), tensor([[180.],
        [142.]])]
Epoch    0/20 Batch 2/3 Cost: 10625.849609
2
[tensor([[93., 88., 93.]]), tensor([[185.]])]
Epoch    0/20 Batch 3/3 Cost: 5451.569824
0
[tensor([[89., 91., 90.],
        [73., 66., 70.]]), tensor([[180.],
        [142.]])]
Epoch    1/20 Batch 1/3 Cost: 937.502686
1
[tensor([[ 96.,  98., 100.],
        [ 73.,  80.,  75.]]), tensor([[196.],
        [152.]])]
Epoch    1/20 Batch 2/3 Cost: 265.944763
2
[tensor([[93., 88., 93.]]), tensor([[185.]])]
Epoch    1/20 Batch 3/3 Cost: 244.422516
0
[tensor([[ 96.,  98., 100.],
        [ 73.,  80.,  75.]]), tensor([[196.],
        [152.]])]
Epoch    2/20 Batch 1/3 Cost: 3.523164
1
[tensor([[93., 88., 93.],
        [73., 66., 70.]]), tensor([[185.],
        [142.]])]
Epoch    2/20 Batch 2/3 Cost: 55.838959
2
[tensor([[89., 91., 

In [13]:
# 임의의 입력 [73, 80, 75]를 선언
new_var =  torch.FloatTensor([[73, 80, 75]]) 
# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) 
print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", pred_y) 

훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[157.2804]], grad_fn=<AddmmBackward>)


In [14]:
print(list(model.parameters()))

[Parameter containing:
tensor([[0.2419, 1.1128, 0.6705]], requires_grad=True), Parameter containing:
tensor([0.3028], requires_grad=True)]


---